# Hypothesis to test:
### Removing objective sentences from reviews helps predict star rating from reviews

In [39]:
import numpy as np
import pandas as pd
import pickle
import gzip
import math
from IPython.display import Markdown, display
from sklearn.feature_extraction.text import TfidfVectorizer

# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

# %autosave 50

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configuration

In [43]:
# Subsampling from Amazon reviews
NB_SAMPLES = 2000  # up to 200k, then change the input file

# Gradient Boosting Classifier parameters
N_TREES = math.floor(np.sqrt(NB_SAMPLES))
LEARN_RATE = 0.01
MIN_IN_LEAF = 3 #7
N_TREES

44

## Get user comments

In [3]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path, trunc=0):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1        
    if trunc > 0 and i > trunc: 
        break
  return pd.DataFrame.from_dict(df, orient='index')

In [4]:
data_path = '../../datasets/'
file_name = 'reviews_Movies_and_TV.json.gz'

In [5]:
# comments_df = getDF(data_path + file_name, 200000)
# comments_df.loc[0,'reviewText']
# print(comments_df.shape)
# comments_df.head(1)

In [6]:
# Save the records
# import pickle
# pickle_out = open(data_path + "amzn_200k.pickle","wb")
# pickle.dump(comments_df, pickle_out)
# pickle_out.close()

In [7]:
# Reads records
import pickle
pickle_in = open(data_path + "amzn_200k.pickle","rb")
comments_df = pickle.load(pickle_in)

## Subsample

In [38]:
small = comments_df.loc[:NB_SAMPLES, :]
print(len(small.index))
small.head(3)

2001


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3R5OBKS7OM2IR,0000143502,Rebecca L. Johnson,"[0, 0]",This has some great tips as always and is help...,5.0,Alton... nough said,1358380800,"01 17, 2013"
1,A3R5OBKS7OM2IR,0000143529,Rebecca L. Johnson,"[0, 0]",This is a great pastry guide. I love how Alto...,5.0,Ah Alton...,1380672000,"10 2, 2013"
2,AH3QC2PC1VTGP,0000143561,Great Home Cook,"[2, 4]",I have to admit that I am a fan of Giada's coo...,2.0,Don't waste your money,1216252800,"07 17, 2008"


## Star rating for base case, based on tf-idf

In [42]:
all_tfidf_vectorizer = TfidfVectorizer(max_df=0.98) # at 0.98 or 0.99, same result as 1: 
all_comments_tfidf = all_tfidf_vectorizer.fit_transform(small['reviewText'])
vocab = all_tfidf_vectorizer.vocabulary_
print(len(vocab))

13219


## Star rating modeling for base case, based on tf-idf

## Split comments into separate sentences

In [9]:
from nltk.tokenize import sent_tokenize
small['sentence'] = small['reviewText'].map(sent_tokenize)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
small.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime', 'sentence'],
      dtype='object')

In [11]:
small.drop(['reviewerName', 'helpful', 'reviewText', 'summary', 
            'unixReviewTime', 'reviewTime'], axis=1, inplace=True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
small.shape

(2001, 4)

In [13]:
sentences = small['sentence'] \
.apply(pd.Series) \
.merge(small, left_index = True, right_index = True) \
.drop(['sentence'], axis = 1) \
.melt(id_vars = ['reviewerID', 'asin','overall'], value_name = 'sentence') \
.drop(['variable'], axis = 1) \
.dropna()

print(sentences.shape)
sentences.head(3)

(11115, 4)


,reviewerID,asin,overall,sentence
0,A3R5OBKS7OM2IR,0000143502,5.0,This has some great tips as always and is help...
1,A3R5OBKS7OM2IR,0000143529,5.0,This is a great pastry guide.
2,AH3QC2PC1VTGP,0000143561,2.0,I have to admit that I am a fan of Giada's coo...


## Sentence-level prep & cleaning

In [14]:
%reload_ext autoreload
from utils import split_n_lower, not_about_support

### Split into words and lower the case

In [15]:
sentences['words'] = sentences['sentence'].apply(lambda s: split_n_lower(s))

In [16]:
print(sentences.shape)
sentences.head(3)

(11115, 5)


,reviewerID,asin,overall,sentence,words
0,A3R5OBKS7OM2IR,0000143502,5.0,This has some great tips as always and is help...,"[this, has, some, great, tips, as, always, and..."
1,A3R5OBKS7OM2IR,0000143529,5.0,This is a great pastry guide.,"[this, is, a, great, pastry, guide, .]"
2,AH3QC2PC1VTGP,0000143561,2.0,I have to admit that I am a fan of Giada's coo...,"[i, have, to, admit, that, i, am, a, fan, of, ..."


### Keep support-related sentences as they probably have impact on rating

In [17]:
# on_movies_filter = [not_about_support(word) for word in sentences['words']]
sentences_on_movie = sentences #[on_movies_filter]

print('Removing {} records'.format(sentences.shape[0]- sentences_on_movie.shape[0]))

Removing 0 records


In [18]:
sentences_on_movie.shape

(11115, 5)

### Base case: A reviews with objective and subjective sentences

In [19]:
# Kernel dies here at 50K samples
all_reviews_groups = sentences_on_movie.groupby(['reviewerID','asin'])


In [20]:
all_reviews_stars = all_reviews_groups['overall'].mean()
all_reviews_stars[:3]

reviewerID      asin      
A100CQXJ6D44T9  0005119367    5.0
A100Z2S0880G9A  0005019281    5.0
A102Z4PIK7CYD8  030714139X    5.0
Name: overall, dtype: float64

In [21]:
all_reviews_comments = all_reviews_groups['words'].sum()
print(sentences_on_movie.iloc[0, 4])
print(all_reviews_comments.shape)
print(all_reviews_comments[0])
len(all_reviews_comments)

['this', 'has', 'some', 'great', 'tips', 'as', 'always', 'and', 'is', 'helping', 'me', 'to', 'complete', 'my', 'good', 'eats', 'collection', '.']
(2000,)
['awesome', 'movie', 'and', 'great', 'story', '.', 'highly', 'recommend', 'this', 'movie', 'and', 'follows', 'closely', 'to', 'biblical', 'accounts', '.', 'the', 'message', 'is', 'strong', 'about', 'forgiveness', 'and', 'faithfulness', '.']


2000

### Remove objective sentences for case B using obj-subj model

In [22]:
# pd.set_option('display.max_colwidth', -1)
# sentences_on_movie['sentence']
sentences_on_movie.shape

(11115, 5)

In [23]:
sentences_on_movie.head(2)

,reviewerID,asin,overall,sentence,words
0,A3R5OBKS7OM2IR,0000143502,5.0,This has some great tips as always and is help...,"[this, has, some, great, tips, as, always, and..."
1,A3R5OBKS7OM2IR,0000143529,5.0,This is a great pastry guide.,"[this, is, a, great, pastry, guide, .]"


#### Vectorize along the word space of the obj-subj training set

In [25]:
tfidf = pickle.load(open('pickles/Obj-Subj_tfidf.pkl', 'rb'))
len(tfidf.vocabulary_)

20893

In [35]:
sentences_tfidf = tfidf.transform(sentences_on_movie['sentence']).todense()

In [36]:
sentences_tfidf.shape

(11115, 20893)

#### Apply the obj-subj model

In [29]:
N_TREES = 100
LEARN_RATE = 0.1
MIN_IN_LEAF = 10
pickle_in = open('pickles/GBC_'+ str(N_TREES) +'_' + str(LEARN_RATE) 
                        +'_' + str(MIN_IN_LEAF) + '_20min.pkl', 'rb')
gb_model = pickle.load(pickle_in)

In [30]:
y_test = gb_model.predict(sentences_tfidf)
len(y_test)

11115

In [ ]:
subjective_sentences = sentences_on_movie[y_test == 1]

In [34]:
display(Markdown('### Removing {} objective sentences'
                 .format(len(y_test) - len(subjective_sentences))))

### Removing 3340 objective sentences

In [ ]:
subjective_sentences.head(2)

#### Merge the sentences back into paragraph reviews

In [ ]:
subj_groups = subjective_sentences.groupby(['reviewerID','asin'])
subj_reviews_stars = subj_groups['overall'].mean()
# subjective_reviewssubjective_reviews['sentence'].apply(lambda x: x.sum())
# subjective_reviews_reviews = 
subj_reviews_stars[:3]

In [ ]:
subj_review_comments = subj_groups['words'].sum()
print(subj_review_comments.shape)
print(subj_review_comments[0])
subj_review_comments[:3]

### Check that stars still correspond to the right movie

In [ ]:
start = 6000
end = 6010
all_reviews_comments.loc[('A33Z7JTV7SSW9Y', '0718000315')]


In [ ]:
print(all_reviews_stars.loc[('A33Z7JTV7SSW9Y', '0718000315')])
print(sentences_on_movie.loc[sentences_on_movie['reviewerID']=='A33Z7JTV7SSW9Y']) 
# and sentences_on_movie['asin']=='0718000315'])

In [ ]:
pd.options.display.max_colwidth = -1
print(small.loc[small['reviewerID']=='A33Z7JTV7SSW9Y'])

In [ ]:
sentences_on_movie[start:end]

## Create emotion vectors

In [ ]:
print('Total number of reviews:', all_reviews_comments.shape[0])
print('Total number of subjective reviews:', subj_review_comments.shape[0])

In [ ]:
%reload_ext autoreload
from emotions_seven import Emotions7
emote = Emotions7()

In [ ]:
all_reviews_emotions = emote.vectorize(all_reviews_comments)
print(all_reviews_emotions.shape)

In [ ]:
# emote.emotions_in_text

In [ ]:
all_reviews_emotions.shape

In [ ]:
# all_revs_with_emotions = all_reviews_emotions[emote.emotions_in_text == True]

In [ ]:
# print(all_revs_with_emotions.shape)
# all_revs_stars = all_reviews_stars[emote.emotions_in_text]
all_reviews_emotions[0]

In [ ]:
subj_reviews_emotions = emote.vectorize(subj_review_comments)
print(subj_reviews_emotions.shape)
subj_reviews_emotions[0]

## Fit a model on base case (all comments) for star rating prediction

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(
    all_reviews_emotions, all_reviews_stars, test_size=0.2, random_state=0)

In [ ]:
X_train.shape

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier

# loss: deviance: logistic log likelihood

In [ ]:
gbc_all = GradientBoostingClassifier(learning_rate=LEARN_RATE, 
                                n_estimators=N_TREES, 
                                min_samples_leaf=MIN_IN_LEAF,
                                random_state=0)
gbc_all.fit(X_train, y_train)

In [ ]:
gbr_all = GradientBoostingRegressor(learning_rate=LEARN_RATE, 
                                n_estimators=N_TREES, 
                                min_samples_leaf=MIN_IN_LEAF,
                                random_state=0)
gbr_all.fit(X_train, y_train)

In [ ]:
X_subj_train, X_subj_cv, y_subj_train, y_subj_cv = train_test_split(
    subj_reviews_emotions, subj_reviews_stars, test_size=0.2, random_state=0)
X_subj_train.shape

In [ ]:
gbc_subj = GradientBoostingClassifier(learning_rate=LEARN_RATE, 
                                n_estimators=N_TREES, 
                                min_samples_leaf=MIN_IN_LEAF,
                                random_state=0)
gbc_subj.fit(X_subj_train, y_subj_train)

In [ ]:
print('Gradient Boosting Classifier')
print('Training score using all comments: {0:.2f}'
      .format(gbc_all.score(X_train, y_train)))
print('CV score using all comments: {0:.2f}'
      .format(gbc_all.score(X_cv, y_cv)))
print('')

# print('Training score using subjective comments only: {0:.2f}'
#       .format(gbc_subj.score(X_subj_train, y_subj_train)))
# print('CV score using subjective comments only: {0:.2f}'
#       .format(gbc_subj.score(X_subj_cv, y_subj_cv)))


In [ ]:
print('Gradient Boosting Regressor')
print('Training score using all comments: {0:.2f}'
      .format(gbr_all.score(X_train, y_train)))
print('CV score using all comments: {0:.2f}'
      .format(gbr_all.score(X_cv, y_cv)))
print('')

In [ ]:
np.unique(gbc_subj.predict(X_subj_train))

## Other techniques

In [ ]:
# from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression(random_state=0, solver='lbfgs',
#                        multi_class='multinomial',max_iter=1000)
# lr.fit(X_subj_train, y_subj_train)
# print(lr.score(X_subj_train, y_subj_train))
# print(lr.score(X_subj_cv, y_subj_cv))

In [ ]:
import statsmodels.api as sm
ols_all = sm.OLS(y_train, X_train)
results_all = ols_all.fit()
results_all.summary()

In [ ]:
ols_subj = sm.OLS(y_subj_train, X_subj_train)
results_subj = ols_subj.fit()
results_subj.summary()

In [ ]:
import seaborn as sns

all_reviews_emotions, all_reviews_stars

sns.heatmap(raw_df.corr(), annot=True)